# Creación y Almacenamiento de Embeddings en Chroma DB

En el presente Notebook se crearán la documentación base del esquema reducido con el que trabajaremos, basada en el `information_schema` correspondiente, que luego se nutrirá por fuera con mayor información relevante para negocio. 

Adicionalmente, por fuera del Notebook, se confeccionará documentación adicional, como reglas de negocio y few-shot examples para, finalmente, obtener chunks de todos estos documentos y crear y almacenar los mismos, con su metadata asociada, en una base de datos vectorial de Chroma DB.

## Inicialización

### Librerías



In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import yaml
import json
from IPython.display import Markdown

from enum import Enum
from collections import defaultdict, OrderedDict
import pandas as pd

import uuid
import chromadb
from google import genai

from langchain.embeddings.base import Embeddings as LangchainEmbeddings
from langchain_chroma import Chroma

from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

load_dotenv('../.env')
pd.set_option('display.max_columns', None)
yaml.add_representer(OrderedDict, lambda dumper, data: dumper.represent_dict(data.items()))

notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from src.pg_sql import execute_query


### Constantes

In [2]:
DATABASE = 'database'
SCHEMAS = 'schemas'
TABLES = 'tables'
COLUMNS = 'columns'
DESCRIPTION = 'description'
NAME = 'name'
DATA_TYPE = 'data_type'
PRIMARY_KEY = 'is_primary_key'
FOREIGN_KEY = 'is_foreign_key'
REFERENCE = 'reference'
TO_DO = '[To be completed ...]'

OUTPUT_PATH = '../data/embeddings/auxs'
MDL_PATH = '../data/embeddings/documents/MDL_adventure_works_dw.yaml'

CHROMA_DB_PATH = '../data/embeddings/chroma'
TABLES_SUMMARY_COLLECTION_NAME = 'mdl_tables_summary'
COLUMNS_COLLECTION_NAME = 'mdl_columns'

AZURE_OPENAI_EMBEDDING_MODEL = 'text-embedding-3-large'
GENAI_EMBEDDING_MODEL = 'gemini-embedding-001'

### Clase personalizadas

In [23]:
class GenaiEmbedConfigTaskType(Enum):
    """
    Enum representing the different task types for GenAI embeddings.
    The value of each member is the same as its name.
    """
    CLASSIFICATION = 'CLASSIFICATION'
    CLUSTERING = 'CLUSTERING'
    RETRIEVAL_DOCUMENT = 'RETRIEVAL_DOCUMENT'
    RETRIEVAL_QUERY = 'RETRIEVAL_QUERY'
    QUESTION_ANSWERING = 'QUESTION_ANSWERING'
    FACT_VERIFICATION = 'FACT_VERIFICATION'
    CODE_RETRIEVAL_QUERY = 'CODE_RETRIEVAL_QUERY'
    SEMANTIC_SIMILARITY = 'SEMANTIC_SIMILARITY'



class GenAIExtendedEmbeddingFunction(LangchainEmbeddings):
    def __init__(self, model: str):
        """
        Initializes the class with the Gemini model.
        
        Args:
            model (str): The name of the Gemini model.
        """
        self.model = model
        self.genai_client = genai.Client()
    
    def _get_genai_embeddings(self, contents: list[str], task_type: GenaiEmbedConfigTaskType | None = None) -> list[list[float]]:
        """
        Internal method to get embeddings from the GenAI API.
        """
        embed_config = genai.types.EmbedContentConfig(task_type=task_type.value) if task_type else None
        
        response = self.genai_client.models.embed_content(
            model= self.model,
            contents= contents,
            config= embed_config
        )
        
        return [embed.values for embed in response.embeddings]
        
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """
        Generates embeddings for a list of documents.
        """
        return self._get_genai_embeddings(
            contents= texts,
            task_type= GenaiEmbedConfigTaskType.RETRIEVAL_DOCUMENT
        )

    def embed_query(self, text: str) -> list[float]:
        """
        Generates an embedding for a single query string.
        """
        return self._get_genai_embeddings(
            contents= [text],
            task_type= GenaiEmbedConfigTaskType.RETRIEVAL_QUERY
        )[0]

### Funciones aplicables a todos los documentos

In [4]:
def add_docs_to_chroma_col(chunks: list[dict], collection: Chroma) -> list[str]:
    """
    Store a list of document chunks into a ChromaDB collection using langchain.

    Each chunk is expected to be a dictionary containing 'content' (text of the chunk)
    and 'metadata' (associated metadata) keys. The function generates unique IDs and, if required,
    computes embeddings for each document chunk before adding them to the collection.

    Args:
        chunks (list[dict]): A list of dictionaries, each with keys:
                             - 'content' (str): The text content of the chunk.
                             - 'metadata' (dict): Metadata associated with the chunk.
        collection (langchain_chroma.Chroma): The target ChromaDB collection by langchain to store the data in.

    Returns:
        ids[list[str]]
    """

    ids = [str(uuid.uuid4()) for _ in chunks]

    collection.add_documents(
        ids = ids,
        documents= chunks
    )



def cosine_distance_relevance_score_fn(distance: float) -> float:
    """Normalize the distance [0, 2] to a score on a scale [0, 1]."""
    return 1.0 - distance / 2

## Fichero YAML con MDL del esquema de interés

### Obtener fichero base desde `information_schema`

Primero obtendremos un fichero base construido utilizando la query `/data/embeddings/auxs/get_information_schema.sql`, sobre el que luego se añadirá metadata extra. Para esto, definimos algunas funciones que nos serán de utilidad:

In [4]:
def get_information_schema(query_path: str, db_names_list: list[str], schema_names_list: list[str]) -> str:
    """
    Reads an SQL query from a file and replaces placeholder lists with formatted strings.

    This function is designed to work with SQL queries that have specific placeholders
    for database and schema names. It reads the query from the given file path, 
    formats the input lists of names into a single quoted, comma-separated string, 
    and replaces the placeholders in the query.

    Args:
        query_path (str): The file path to the SQL query. The query should
                          contain the placeholders `[db_names_list]` and
                          `[schema_names_list]`.
        db_names_list (list[str]): A list of database names to be formatted
                                   and inserted into the query.
        schema_names_list (list[str]): A list of schema names to be formatted
                                       and inserted into the query.

    Returns:
        str: The complete SQL query with the placeholders replaced by
             the formatted database and schema names.

    Raises:
        FileNotFoundError: If the specified query_path does not exist.
        
    Example:
        >>> from pathlib import Path
        >>> # Assume 'my_query.sql' contains:
        >>> # SELECT * FROM information_schema.tables WHERE table_schema IN ([schema_names_list])
        >>> # And we create a dummy file for the example:
        >>> Path('my_query.sql').write_text("SELECT * FROM information_schema.tables WHERE table_schema IN ([schema_names_list])")
        >>> db_list = ['db1', 'db2']
        >>> schema_list = ['schema_a', 'schema_b']
        >>> get_information_schema('my_query.sql', db_list, schema_list)
        "SELECT * FROM information_schema.tables WHERE table_schema IN ('schema_a', 'schema_b')"
    """
    query = Path(query_path).read_text()

    db_names = "'" + "', '".join(db_names_list) + "'"
    schema_names = "'" + "', '".join(schema_names_list) + "'"

    return query.replace('[db_names_list]', db_names).replace('[schema_names_list]', schema_names)



def format_yaml(yaml_str: str) -> str:
    """
    Formats a YAML string by adding a blank line before each list item
    that isn't preceded by a list key.
    """
    last_line = ''
    last_line_list_init = False
    last_line_empty = False

    lines = list()

    for line in yaml_str.split('\n'):
        if line.strip().startswith('-') and not last_line_list_init and not last_line_empty:
            last_line += '\n'

        lines.append(last_line)
        last_line = line
        last_line_list_init = last_line.strip().endswith(':')
        last_line_empty = last_line.strip()==''

    lines.append(line)

    return '\n'.join(lines)

In [5]:
GET_INFORMATION_SCHEMA_SQL = '../data/embeddings/auxs/get_information_schema.sql'
DB_NAME = 'adventure_works_dw'
SCHEMA_NAME = 'sales'


information_schema_data = execute_query(get_information_schema(
    query_path= GET_INFORMATION_SCHEMA_SQL,
    db_names_list= [DB_NAME],
    schema_names_list= [SCHEMA_NAME]
))

Veamos el aspecto que tienen los resultados de nuestra query:

In [6]:
information_schema_data

[{'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'customer_key',
  'column_type': 'INT4',
  'primary_key': True,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'geography_key',
  'column_type': 'INT4',
  'primary_key': False,
  'foreign_key': True,
  'target': 'sales.dim_geography.geography_key'},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'customer_full_name',
  'column_type': 'TEXT',
  'primary_key': False,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'birth_date',
  'column_type': 'DATE',
  'primary_key': False,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
 

Lo convertimos en un Data Frame de Pandas para que sea más vistoso:

In [7]:
pd.DataFrame(information_schema_data)

,db_name,schema_name,table_name,column_name,column_type,primary_key,foreign_key,target
0,adventure_works_dw,sales,dim_customer,customer_key,INT4,True,False,None
1,adventure_works_dw,sales,dim_customer,geography_key,INT4,False,True,sales.dim_geography.geography_key
2,adventure_works_dw,sales,dim_customer,customer_full_name,TEXT,False,False,None
3,adventure_works_dw,sales,dim_customer,birth_date,DATE,False,False,None
4,adventure_works_dw,sales,dim_customer,marital_status,BPCHAR(1),False,False,None
...,...,...,...,...,...,...,...,...
107,adventure_works_dw,sales,fact_sales,freight,NUMERIC,False,False,None
108,adventure_works_dw,sales,fact_sales,order_date,DATE,False,False,None
109,adventure_works_dw,sales,fact_sales,due_date,DATE,False,False,None
110,adventure_works_dw,sales,fact_sales,ship_date,DATE,False,False,None


Ahora procederemos a crear el YAML base con el MDL de nuestro esquema, que podremos tomar como punto de partida para luego añadirle metadata extra manualmente:

In [8]:
dbs_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for row in information_schema_data:
    db_name = row.get('db_name')
    schema_name = row.get('schema_name')
    table_name = row.get('table_name')
    
    dbs_data[db_name][schema_name][table_name].append(row)

for db_name, schemas_data in dbs_data.items():
    db = OrderedDict()
    db[DATABASE] = db_name
    db[DESCRIPTION] = TO_DO

    schemas = list()
    for schema_name, tables_data in schemas_data.items():
        schema = OrderedDict()
        schema[NAME] = schema_name
        schema[DESCRIPTION] = TO_DO

        tables = list()
        for table_name, columns_data in tables_data.items():
            table = OrderedDict()
            table[NAME] = table_name
            table[DESCRIPTION] = TO_DO

            columns = list()
            for column_data in columns_data:
                column = OrderedDict()
                column[NAME] = column_data.get('column_name')
                column[DESCRIPTION] = TO_DO
                column[DATA_TYPE] = column_data.get('column_type')
                
                if column_data.get('primary_key'):
                    column[PRIMARY_KEY] = True

                if column_data.get('foreign_key'):
                    column[FOREIGN_KEY] = True
                    column[REFERENCE] = column_data.get('target')

                columns.append(column)

            table[COLUMNS] = columns
            tables.append(table)
        
        schema[TABLES] = tables
        schemas.append(schema)
    
    db[SCHEMAS] = schemas

    mdl_file_path = f'{OUTPUT_PATH}/MDL_{db_name}.yaml'
    with open(mdl_file_path, 'w') as mdl:
        mdl.write(format_yaml(yaml.dump(db)))

    print(f'>  Fichero MDL base almacenado en {mdl_file_path}')

>  Fichero MDL base almacenado en ../data/embeddings/auxs/MDL_adventure_works_dw.yaml


### Creación de embeddings

#### Chunking

Tomando como base el fichero obtenido en el apartado anterior, se ha creado un fichero que del MDL de nuestro esquema de interés que ha sido nutrido con metadata adicional, como descripciones para cada tabla y campo. Este fichero se encuentra en la ruta `/data/embeddings/documents/MDL_adventure_works_dw.yaml`.

Procederemos ahora a procesar este fichero para crear chunks de la información de cada una de las tablas y almacenar sus embeddings en nuestra base de datos vectorial de `chroma_db`, persistida en la ruta `/data/embeddings/chroma_db/`.

Procedemos ahora a definir funciones que nos permitirán chunkear el documento mdl desde un enfoque más integral a nivel de tablas, como también a un enfoque más granular a nivel de columnas:

In [5]:
def chunk_mdl_by_table_summary(mdl_data: dict) -> list[Document]:
    """
    Chunks a Model-Driven Library (MDL) data structure into a list of Documents,
    with each document representing a summary of a single table.

    Each Document's page_content contains a summary of the table, including its
    database, schema, table name, description, primary keys, and foreign keys.
    The Document's metadata includes the database, schema, and table name for
    easy lookup and filtering.

    Args:
        mdl_data: A dictionary representing the MDL data structure. It is expected
                  to contain keys for 'database', 'schemas', 'tables', 'columns',
                  and other relevant metadata.

    Returns:
        A list of Document objects, where each object holds a table summary chunk
        from the input MDL data.
    """

    chunks = []

    database_name = mdl_data[DATABASE]

    for schema in mdl_data[SCHEMAS]:
        schema_name = schema[NAME]

        for table in schema[TABLES]:
            table_name = table[NAME]
            table_description = table[DESCRIPTION]
            table_columns = table[COLUMNS]
            
            chunk_content_list = [
                f'Database: {database_name}',
                f'Schema: {schema_name}',
                f'Table: {table_name}',
                f'Table description: {table_description}'
            ]

            table_primary_key = [col[NAME] for col in table_columns if col.get(PRIMARY_KEY)]
            if table_primary_key:
                chunk_content_list.append('Table PRIMARY KEY:')
                for pk in table_primary_key:
                    chunk_content_list.append(f'- {pk}')

            table_foreign_keys = [{'column_name': col[NAME], 'reference': col[REFERENCE]} for col in table_columns if col.get(FOREIGN_KEY)]
            if table_foreign_keys:
                chunk_content_list.append('Table FOREIGN KEYS (Column name, Reference):')
                for fk in table_foreign_keys:
                    chunk_content_list.append(f'- ({fk["column_name"]}, {fk["reference"]})')

            chunk_content = '\n'.join(chunk_content_list)

            metadata = {
                'database_name': database_name,
                'schema_name': schema_name,
                'table_name': table_name
            }

            chunks.append(Document(page_content= chunk_content, metadata= metadata))

    return chunks



def chunk_mdl_by_column(mdl_data: dict) -> list[Document]:
    """
    Chunks a Model-Driven Library (MDL) data structure into a list of Documents,
    with each document representing a single column.

    This function iterates through the schemas, tables, and columns of the MDL
    data. For each non-key column (excluding primary and foreign keys), it
    creates a Document. The Document's page_content contains the column's
    name, data type, and description. The Document's metadata includes
    the database, schema, table, and column details for easy retrieval and
    filtering.

    Args:
        mdl_data: A dictionary representing the MDL data structure. It is expected
                  to contain keys for 'database', 'schemas', 'tables', 'columns',
                  and other relevant metadata.

    Returns:
        A list of Document objects, where each object holds a chunk of data
        for a non-key column from the input MDL data.
    """
    chunks = []

    database_name = mdl_data[DATABASE]

    for schema in mdl_data[SCHEMAS]:
        schema_name = schema[NAME]

        for table in schema[TABLES]:
            table_name = table[NAME]
            table_columns = table[COLUMNS]

            for col in table_columns:
                chunk_content_list = list()

                if col.get(PRIMARY_KEY) or col.get(FOREIGN_KEY):
                    continue

                column_name = col[NAME]
                column_data_type = col[DATA_TYPE]
                column_description = col[DESCRIPTION]

                chunk_content_list.append(f'Column name: {column_name}')
                chunk_content_list.append(f'Column data type: {column_data_type}')
                chunk_content_list.append(f'Column description: {column_description}')

                chunk_content = '\n'.join(chunk_content_list)

                metadata = {
                    'database_name': database_name,
                    'schema_name': schema_name,
                    'table_name': table_name,
                    'column_name': column_name,
                    'column_data_type': column_data_type
                }

                chunks.append(Document(page_content= chunk_content, metadata= metadata))

    return chunks


Veamos ahora el aspecto que tiene, por ejemplo, el chunk de mayor tamaño para cada uno de los tipos:

In [6]:
with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)

chunks = chunk_mdl_by_table_summary(mdl_data)

max_len = max([len(c.page_content) for c in chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in chunks if len(c.page_content) == max_len][0])

max_len=1020

Database: adventure_works_dw
Schema: sales
Table: fact_sales
Table description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.
Table PRIMARY KEY:
- sales_order_number
- sales_order_line_number
Table FOREIGN KEYS (Column name, Reference):
- (product_key, sales.dim_product.product_key)
- (reseller_key, sales.dim_reseller.reseller_key)
- (employee_key, sales.dim_sales_person.employee_key)
- (customer_key, sales.dim_customer.customer_key)
- (promotion_key, sales.dim_promotion.promotion_key)
- (sales_territory_key, sales.dim_sa

In [7]:
with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)

chunks = chunk_mdl_by_column(mdl_data)

max_len = max([len(c.page_content) for c in chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in chunks if len(c.page_content) == max_len][0])

max_len=265

Column name: sale_source
Column data type: TEXT
Column description: Indicador de la fuente por la que ha sido realizado el pedido. reseller_sales=Pedido realizado por un vendedor para una tienda/distribuidor, internet_sales=Pedido realizado en línea por un cliente.

{'database_name': 'adventure_works_dw', 'schema_name': 'sales', 'table_name': 'fact_sales', 'column_name': 'sale_source', 'column_data_type': 'TEXT'}


#### LangChain: Chroma DB + Azure OpenAI Embeddings

Ahora haremos una primera implementación de prueba de Chroma valiéndonos de funcionalidades de la librería LangChain, usando un modelo de embeddings de OpenAI a través de Azure AI Foundry.

El primer paso es crear un cliente persistente de Chroma DB, e inicializar las colecciones tanto a nivel de tablas como de columnas. Al inicializar las colecciones, especificaremos que queremos usar como métrica de distancia la distancia de Coseno, que se calcula de la siguiente manera:

$$Distancia = 1 - \frac{A \cdot B}{\|A\| \|B\|}$$

Donde:
* $A \cdot B$ es el producto punto (producto escalar) de los vectores $A$ y $B$.
* $∥A∥$ y $∥B∥$ son las magnitudes (o normas euclidianas) de los vectores $A$ y $B$, respectivamente.
​

La métrica **Distancia del Coseno** mide solo el ángulo entre vectores, lo que lo hace ideal para embeddings de texto, y particularmente para modelos de embeddings como los de [OpenAI cuyos valores vienen normalizados](https://help.openai.com/en/articles/8984345-which-distance-function-should-i-use).

Como es fácil ver en su expresión, esta métrica de distancia devuelve valores en el rango $[0, 2]$. Cuanto más cerca de 0, más similares serán los vectores $A$ y $B$. Esto nos deja en la necesidad de definir una métrica de `relevance_score` personalizada, que nos permita reescalar los resultados en valores comprendidos entre $[0, 1]$, siendo los valores más cercanos a 1 indicativo de mayor similitud entre los vectores. Esta métrica la hemos definido en la función `cosine_distance_relevance_score_fn` de la siguiente manera:

$$R_{score} = 1 - \frac{Distancia}{2}$$



In [8]:
test_openai_embeddings_path = '../data/embeddings/tests/openai'
chroma_client = chromadb.PersistentClient(path= test_openai_embeddings_path)
openai_embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)


tables_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if tables_summary_chunks:
    try:
        chroma_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    tables_summary_collection = Chroma(
        client= chroma_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= openai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(tables_summary_chunks, tables_summary_collection)


columns_chunks = chunk_mdl_by_column(mdl_data)
if columns_chunks:
    try:
        chroma_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    columns_collection = Chroma(
        client= chroma_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= openai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(columns_chunks, columns_collection)


Lanzamos ahora una consulta a cada una de las colecciones creadas:

In [9]:
test_openai_embeddings_path = '../data/embeddings/tests/openai'
chroma_client = chromadb.PersistentClient(path= test_openai_embeddings_path)
openai_embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)

tables_summary_collection = Chroma(
    client= chroma_client,
    collection_name= TABLES_SUMMARY_COLLECTION_NAME,
    embedding_function= openai_embeddings,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    create_collection_if_not_exists= False
)

columns_collection = Chroma(
    client= chroma_client,
    collection_name= COLUMNS_COLLECTION_NAME,
    embedding_function= openai_embeddings,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    create_collection_if_not_exists= False
)

In [10]:
QUERY = 'Los 10 artículos más comprados.'

In [11]:
tables_summary_results = tables_summary_collection.similarity_search_with_relevance_scores(
    query= QUERY,
    k= 10,
    score_threshold= 0.20
)

relevant_columns = []
for table_result in tables_summary_results:
    database_name = table_result[0].metadata['database_name']
    schema_name = table_result[0].metadata['schema_name']
    table_name = table_result[0].metadata['table_name']

    table_filter = {
        "$and": [
            {"database_name": {"$eq": database_name}},
            {"schema_name": {"$eq": schema_name}},
            {"table_name": {"$eq": table_name}}
        ]
    }

    columns_results = columns_collection.similarity_search_with_relevance_scores(
        query= QUERY,
        k= 15,
        filter=table_filter,
        score_threshold= 0.15
    )

    if columns_results:
        relevant_columns.append({
            'table_summary': {
                'content': table_result[0].page_content,
                'relevance_score': table_result[1]
            },
            'columns': [{'content': col[0].page_content, 'relevance_score': col[1]} for col in columns_results]
        })

if not relevant_columns:
    relevant_columns = [f'No docs retrieved for the query "{QUERY}"']

print(json.dumps(relevant_columns, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.15


[
  {
    "table_summary": {
      "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promot

In [12]:
print(relevant_columns)

[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (sa

In [13]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. When it is possible, match the language of the query fields to the user's query. For example, if user's query is in spanish, use `spanish_...`.
8. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query. Don't suggest any query, only bring information in plain text. 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(relevant_columns, ensure_ascii=False)
})

Markdown(contexto_resumido)

Para obtener información sobre los 10 artículos más comprados, se pueden considerar las siguientes tablas y columnas relevantes de la base de datos `adventure_works_dw`:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con granularidad a nivel de línea.
   - **Claves Primarias**: 
     - `sales_order_number`
     - `sales_order_line_number`
   - **Columnas relevantes**:
     - `order_quantity`: Cantidad vendida de cada producto.
     - `product_key`: Clave del producto (clave foránea que se relaciona con la tabla `dim_product`).

2. **Tabla: dim_product**
   - **Descripción**: Almacena información detallada de cada producto vendido.
   - **Clave Primaria**: 
     - `product_key`
   - **Columnas relevantes**:
     - `english_product_name`: Nombre del producto (en inglés).
     - `spanish_product_category_name`: Categoría principal a la que pertenece el producto (en español).

Para determinar los artículos más comprados, se puede utilizar la columna `order_quantity` de la tabla `fact_sales` junto con la clave del producto para hacer referencia a la tabla `dim_product` y obtener los nombres de los productos.

#### LangChain: Chroma DB + Google GenAI Embeddings

Ahora haremos una segunda implementación de prueba de Chroma valiéndonos de funcionalidades de la librería LangChain, usando un modelo de embeddings de Google Gemini a través de Gemini API, pero trabajando con VertexAI de fondo, lo que nos permite asegurar que los datos permanezcan en la region / location deseada.

Dado que la clase implementada por LangChain para utilizar embeddings de Gemini API no soporta usar VertexAI por detrás, hemos procedido a implementar una clase personalizada que sí nos los permite, a la que hemos llamado `GenAIExtendedEmbeddingFunction`.

Generamos una instancia de la clase y verificamos que los parámetros son los esperados:

In [15]:
genai_embeddings = GenAIExtendedEmbeddingFunction(model= GENAI_EMBEDDING_MODEL)
print(f'VertexAI: {genai_embeddings.genai_client._api_client.vertexai}')
print(f'Project: {genai_embeddings.genai_client._api_client.project}')
print(f'Location: {genai_embeddings.genai_client._api_client.location}')
print(f'Auth Tokens: {genai_embeddings.genai_client.auth_tokens}')

VertexAI: True
Project: gen-lang-client-0151767776
Location: europe-west1
Auth Tokens: <google.genai.tokens.Tokens object at 0x0000027B349EAB90>


El próximo paso es crear un cliente persistente de Chroma DB, e inicializar las colecciones tanto a nivel de tablas como de columnas. Al inicializar las colecciones, especificaremos que queremos usar como métrica de distancia la distancia de Coseno, que se calcula de la siguiente manera:

$$Distancia = 1 - \frac{A \cdot B}{\|A\| \|B\|}$$

Donde:
* $A \cdot B$ es el producto punto (producto escalar) de los vectores $A$ y $B$.
* $∥A∥$ y $∥B∥$ son las magnitudes (o normas euclidianas) de los vectores $A$ y $B$, respectivamente.
​

La métrica **Distancia del Coseno** mide solo el ángulo entre vectores, lo que lo hace ideal para embeddings de texto, y particularmente para modelos de embeddings como el que usaremos [(`gemini-embedding-001`) cuyos valores vienen normalizados](https://ai.google.dev/gemini-api/docs/embeddings#quality-for-smaller-dimensions).

Como es fácil ver en su expresión, esta métrica de distancia devuelve valores en el rango $[0, 2]$. Cuanto más cerca de 0, más similares serán los vectores $A$ y $B$. Esto nos deja en la necesidad de definir una métrica de `relevance_score` personalizada, que nos permita reescalar los resultados en valores comprendidos entre $[0, 1]$, siendo los valores más cercanos a 1 indicativo de mayor similitud entre los vectores. Esta métrica la hemos definido en la función `cosine_distance_relevance_score_fn` de la siguiente manera:

$$R_{score} = 1 - \frac{Distancia}{2}$$

In [ ]:
test_genai_embeddings_path = '../data/embeddings/tests/genai'
chroma_client = chromadb.PersistentClient(path= test_genai_embeddings_path)
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)


tables_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if tables_summary_chunks:
    try:
        chroma_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    tables_summary_collection = Chroma(
        client= chroma_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(tables_summary_chunks, tables_summary_collection)


columns_chunks = chunk_mdl_by_column(mdl_data)
if columns_chunks:
    try:
        chroma_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    columns_collection = Chroma(
        client= chroma_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(columns_chunks, columns_collection)


Lanzamos ahora una consulta a cada una de las colecciones creadas:

In [17]:
test_genai_embeddings_path = '../data/embeddings/tests/genai'
chroma_client = chromadb.PersistentClient(path= test_genai_embeddings_path)
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

tables_summary_collection = Chroma(
    client= chroma_client,
    collection_name= TABLES_SUMMARY_COLLECTION_NAME,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    embedding_function= genai_embeddings
)

columns_collection = Chroma(
    client= chroma_client,
    collection_name= COLUMNS_COLLECTION_NAME,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    embedding_function= genai_embeddings
)

In [18]:
QUERY = 'Los 10 artículos más comprados.'

In [19]:
tables_summary_results = tables_summary_collection.similarity_search_with_relevance_scores(
    query= QUERY,
    k= 10,
    score_threshold= 0.75
)

relevant_columns = []
for table_result in tables_summary_results:
    database_name = table_result[0].metadata['database_name']
    schema_name = table_result[0].metadata['schema_name']
    table_name = table_result[0].metadata['table_name']

    table_filter = {
        "$and": [
            {"database_name": {"$eq": database_name}},
            {"schema_name": {"$eq": schema_name}},
            {"table_name": {"$eq": table_name}}
        ]
    }

    columns_results = columns_collection.similarity_search_with_relevance_scores(
        query= QUERY,
        k= 15,
        filter=table_filter,
        score_threshold= 0.75
    )

    if columns_results:
        relevant_columns.append({
            'table_summary': {
                'content': table_result[0].page_content,
                'relevance_score': table_result[1]
            },
            'columns': [{'content': col[0].page_content, 'relevance_score': col[1]} for col in columns_results]
        })

if not relevant_columns:
    relevant_columns = [f'No docs retrieved for the query "{QUERY}"']

print(json.dumps(relevant_columns, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.75


[
  {
    "table_summary": {
      "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promot

In [20]:
print(relevant_columns)

[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (sa

In [21]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. When it is possible, match the language of the query fields to the user's query. For example, if user's query is in spanish, use `spanish_...`.
8. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query. Don't suggest any query, only bring information in plain text. 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(relevant_columns, ensure_ascii=False)
})

Markdown(contexto_resumido)

Para obtener información sobre los 10 artículos más comprados, se pueden considerar las siguientes tablas y columnas relevantes de la base de datos `adventure_works_dw` en el esquema `sales`:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con granularidad a nivel de línea.
   - **Claves Primarias**: 
     - `sales_order_number`
     - `sales_order_line_number`
   - **Columnas relevantes**:
     - `order_quantity`: Cantidad vendida de cada producto.
     - `product_key`: Clave del producto (relacionada con la tabla `dim_product`).

2. **Tabla: dim_product**
   - **Descripción**: Almacena información detallada de cada producto vendido.
   - **Clave Primaria**: 
     - `product_key`
   - **Columnas relevantes**:
     - `english_product_name`: Nombre del producto (en inglés).
     - `spanish_product_name`: Nombre del producto (en español).
     - `list_price`: Precio de venta del producto en USD.

Para realizar un análisis sobre los artículos más comprados, se utilizarían principalmente las columnas `order_quantity` de la tabla `fact_sales` y `product_key` para relacionar con la tabla `dim_product`, donde se puede obtener el nombre y precio de los productos.

#### Conclusiones e Implementación Final

Como podemos ver, siguiendo el mismo proceso para ambos modelos de embeddings, el modelo `gemini-embedding-001` de Google, aunque parece devolver resultados similares que los que se obtienen con `text-embedding-3-small` de OpenAI, consigue mejores resultados en términos de relevancia. Esto nos hace pensar que es mejor al capturar la semántica de nuestro documentos frente a la de las consultas.

Esto puede deberse, en gran medida, a que , aunque ambos modelos presentan un **tamaño de embeddings de 3072**, el modelo de Google presenta un rendimiento destacable en tareas multilingües, como puede apreciarse en el [paper de su presentación](https://arxiv.org/html/2503.07891v1). Este es un punto no menor al trabajar nuestra documentación, ya que la misma mezcla conceptos en inglés (nombres de tablas y campos) y en español (descripciones), como asi también al momento de consultar, que podrá hacerse en cualquiera de estos 2 idiomas de manera indistinta.

Teniendo en cuenta estas conclusiones, finalmente, **procedemos a implementar nuestras colecciones de `Chroma DB` de manera definitiva usando el modelo `gemini-embedding-001`**.

In [22]:
chroma_client = chromadb.PersistentClient(path= CHROMA_DB_PATH)
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)


tables_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if tables_summary_chunks:
    try:
        chroma_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    tables_summary_collection = Chroma(
        client= chroma_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(tables_summary_chunks, tables_summary_collection)


columns_chunks = chunk_mdl_by_column(mdl_data)
if columns_chunks:
    try:
        chroma_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    columns_collection = Chroma(
        client= chroma_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(columns_chunks, columns_collection)


## Fichero MarkDown con Reglas de Negocio

## Fichero JSON con Ejemplos de Consultas